<a href="https://colab.research.google.com/github/yonatanrtt/transformers_layers/blob/main/V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !git clone https://github.com/huggingface/transformers.git
# !mv transformers/ transformers_aa/
# !cp -R transformers_aa/src/transformers/ transformers/
# import shutil
# shutil.rmtree('transformers_aa')	

In [ ]:
# !pip install transformers --quiet
# !pip uninstall transformers

In [ ]:
!pip install datasets --quiet
from datasets import load_dataset
data = load_dataset("multi_nli")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer, DataCollatorWithPadding

lm_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(lm_name)
model = AutoModelForSequenceClassification.from_pretrained(lm_name, num_labels=3)
config = AutoConfig.from_pretrained(lm_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
def preprocess_function(_example):    
    txt = _example["premise"] +  tokenizer.sep_token +  _example["hypothesis"]
    return tokenizer(txt, truncation=True)
train = data["train"].map(preprocess_function).shuffle(seed=42)
val = data["validation_matched"].map(preprocess_function).shuffle(seed=42)

  0%|          | 0/392702 [00:00<?, ?ex/s]

  0%|          | 0/9815 [00:00<?, ?ex/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install evaluate
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: hypothesis_binary_parse, genre, pairID, premise, hypothesis, premise_binary_parse, hypothesis_parse, promptID, premise_parse. If hypothesis_binary_parse, genre, pairID, premise, hypothesis, premise_binary_parse, hypothesis_parse, promptID, premise_parse are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/content/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 392702
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
 

Step,Training Loss
500,0.795100
1000,0.609000
1500,0.549800
2000,0.540600
2500,0.536600
3000,0.506600
3500,0.509700
4000,0.490400


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

Step,Training Loss
500,0.795100
1000,0.609000
1500,0.549800
2000,0.540600
2500,0.536600
3000,0.506600
3500,0.509700
4000,0.490400
4500,0.500800


Saving model checkpoint to ./results/checkpoint-4500
Configuration saved in ./results/checkpoint-4500/config.json
Model weights saved in ./results/checkpoint-4500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-4500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-4500/special_tokens_map.json
